In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('../')
from FullParser.ClauseParser import ClauseParser

## Parsing tools and data directories

In [9]:
parser = ClauseParser()
import benepar, spacy
nlp = spacy.load('en_core_web_md')
if spacy.__version__.startswith('2'):
    nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
    nlp.add_pipe("benepar", config={"model": "benepar_en3"})

# Function for quick sentence processing
def nlp_sents(string):
    return list(nlp(string).sents)

# Golden Data file paths
dec_path_golden = "../Annotation/declarative_golden_set.json"
pol_path_golden = ".../Annotation/polar_golden_set.json"
alt_path_golden = "../Annotation/alternative_golden_set.json"
const_path_golden = "../Annotation/constituent_golden_set.json"
adv_path_golden = "../Annotation/adversarials_golden_set.json"
flat_path_golden = "../Annotation/golden_sets_flattened.json"

## Parse Golden sentences 

In [10]:
def parse_flat_golden(filename:str):
    golden_df = pd.read_json(filename, orient = 'index')
    golden_parses = []
    parser_parses = []
    for sent in golden_df.sentence.value_counts().to_dict().keys() :
        parses = [dict(row) for i,row in (golden_df[golden_df.sentence == sent]).iterrows()]
        golden_parses.append(parses)
        sent_doc = nlp(sent)
        parsed_sent = list(sent_doc.sents)[0]
        parser_parses.append(parser.parse_clauses(parsed_sent))
    return (parser_parses, golden_parses)

flat_parsed, flat_golden = parse_flat_golden(flat_path_golden)
adv_parsed, adv_golden = parse_flat_golden(adv_path_golden)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/homebrew/Caskroom/miniconda/base/envs/benepar_env/lib/python3.12/site-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/benepar_env/lib/python3.12/site-packages/torch/distributions/distribution.py:53: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


## Evaluation functions

In [11]:
def get_predicate_string(predicate):
    if len(predicate) ==0:
        return ''
    pred_string = ''
    for item in predicate:
        pred_string += str(item['str']) + ' '
    return pred_string[:-1]


def filter_sentences_idx(filt):
    return [idx for idx in [i for i, e in enumerate(flat_golden) if filt(e)] ]

single_idx = filter_sentences_idx(lambda x: len(x) == 1)
multiple_idx = filter_sentences_idx(lambda x: len(x) > 1)


def compare_data(parsed,golden,feature):
    if feature== 'type':
        return [[any([(gp['clause'] == e['clause'] and  gp['type'] == e['type']) for e in parsed[i]]) for gp in gold]  for i,gold in enumerate(golden)]
    if feature=='predicate':
        return [[any([(get_predicate_string(gp['predicate']) == get_predicate_string(e['predicate'])) for e in parsed[i]]) for gp in gold]  for i,gold in enumerate(golden)]
    return [[any([gp[feature] == e[feature] for e in parsed[i]]) for gp in gold]  for i,gold in enumerate(golden)]


## Single Clause Evaluation

In [16]:
parsed_single = [flat_parsed[i] for i in single_idx]
golden_single = [flat_golden[i] for i in single_idx]

# Clause detection

detected_clauses =  list(map(lambda x: len(x)>0,parsed_single))
print('Detection accuracy', np.mean(detected_clauses))

# clause and type

correct_clauses = compare_data(parsed_single,golden_single,'type')
print('Clause accuracy', np.mean(correct_clauses))

# Predicate detection

correct_predicates = compare_data(parsed_single,golden_single,'predicate')
print('Precicate accuracy',np.mean(correct_predicates))

failed_single_predicates = [golden_single[i][0]['sentence'] for i,e in enumerate(correct_predicates) if (not e[0] and correct_clauses[i])]

correct_types = compare_data(parsed_single,golden_single,'type')
print('Type accuracy',np.mean(correct_predicates))


Detection accuracy 0.9523809523809523
Clause accuracy 0.8783068783068783
Precicate accuracy 0.8994708994708994
Type accuracy 0.8994708994708994


## Multiple Clause Evaluation

In [17]:
parsed_multiple = [flat_parsed[i] for i in single_idx]
golden_multiple = [flat_golden[i] for i in single_idx]

# Detection

detected_multiple = [len(gold) == len(parsed_multiple[i]) for i,gold in enumerate(golden_multiple)]
print('Multiple-clauses detection accuracy', np.mean(correct_clauses))

failed_multiple_detect = [parsed_multiple[i] for i,detect in enumerate(detected_multiple) if not detect]

# clause and type

correct_clauses = compare_data(parsed_multiple,golden_multiple,'type')
print('Multiple-clauses clause accuracy ',np.mean(list(map(np.mean,correct_clauses))))

# Predicate detection

correct_predicates = compare_data(parsed_multiple,golden_multiple,'predicate')
print('Multiple-clauses predicate accuracy',np.mean(list(map(np.mean,correct_predicates))))


Multiple-clauses detection accuracy 0.8783068783068783
Multiple-clauses clause accuracy  0.8783068783068783
Multiple-clauses predicate accuracy 0.8994708994708994


## Overall Evaluation

In [40]:
# Clause detection

detected_clauses = list(map(lambda x: len(x)>0,flat_parsed))
print('Overall detection accuracy', np.mean(detected_clauses))

failed_detects = [gp[0]['sentence'] for i,gp in enumerate(flat_golden) if not detected_clauses[i]]

# clause and type

correct_clauses = compare_data(flat_parsed,flat_golden,'type')
print('Overall clause accuracy', np.mean(list(map(all,correct_clauses))))

failed_clauses = [gp[0]['sentence'] for i,gp in enumerate(flat_golden) if (detected_clauses[i] and  not any(correct_clauses[i]) and all(correct_predicates[i]))] 


# Predicate detection

correct_predicates = compare_data(flat_parsed,flat_golden,'predicate')
print('Overall predicate accuracy', np.mean(list(map(all,correct_predicates))))

failed_preds = [gp[0]['sentence'] for i,gp in enumerate(flat_golden) if (detected_clauses[i] and  not any(correct_predicates[i]))]



Overall detection accuracy 0.9530201342281879
Overall clause accuracy 0.8322147651006712
Overall predicate accuracy 0.8747203579418344


## Adversarial sentences

In [34]:
false_positives = [parse for parse in adv_parsed if len(parse)>0]
false_positive_sentences = [parse[0]['sentence'] for parse in false_positives]

In [39]:
[[fp['predicate'] == [] for fp in ent] for ent in false_positives]

[[False], [False], [True], [False], [True], [False], [False], [False], [False]]

## Failure Analyses

### Some useful fonctions for probing

In [42]:
def replace_brackets(parsed_string):
    # Replace round brackets with square brackets
    replaced_string = parsed_string.replace('(', '[').replace(')', ']')
    # Wrap the string with \begin{forest} and \end{forest}
    final_string = '\\begin{adjustbox}{width=0.8\\linewidth}' + '\\begin{forest} ' + replaced_string + ' \\end{forest}' + '\\end{adjustbox}\\\\'
    return final_string

def copy_latex_parse(sentence):
    ps = list(nlp(sentence).sents)[0]
    return replace_brackets(ps._.parse_string)

# Find parse of sentences matching keywords
def find_parse(string):
    return [parse for parse in flat_parsed if (lambda x: (string in x[0]['sentence']) if len(x) > 0 else False)(parse)]
# Find golden parse of sentences matching keywords
def find_golden_parse(string):
    return [gp for gp in flat_golden if (string in gp[0]['sentence'])]

import nltk
def nlp_parse(sent):
    return nltk.Tree.fromstring(list(nlp(sent).sents)[0]._.parse_string).pretty_print()

In [52]:
# View the golden parses matching query
fail_idx = 0
print('Golden',[(gp['predicate'], gp['clause']) for gp in find_golden_parse(failed_preds[fail_idx])[0]])
print('\n')
print('Parsed',[(p['predicate'],p['clause']) for p in find_parse(failed_preds[fail_idx])[0]])

parser.parse_clauses(nlp_sents(failed_preds[fail_idx])[0])
nlp_parse(failed_preds[fail_idx])

Golden [([{'str': 'comment', 'lemma': 'comment', 'POS': 'VERB'}, {'str': 'on', 'lemma': 'on', 'POS': 'ADP'}], 'how'), ([{'str': 'comment', 'lemma': 'comment', 'POS': 'VERB'}, {'str': 'on', 'lemma': 'on', 'POS': 'ADP'}], 'what he is planning')]


Parsed [([{'str': 'comment', 'lemma': 'comment', 'POS': 'VERB'}, {'str': 'on', 'lemma': 'on', 'POS': 'ADP'}, {'str': 'planning', 'lemma': 'plan', 'POS': 'VERB'}], 'how or what he is planning')]
                                                     S                                                          
  ___________________________________________________|________________________________________________________   
 |   |                                    VP                                                                  | 
 |   |                 ___________________|________________                                                   |  
 |   |                |                         |          VP                                             

In [47]:
gold_clauses = []
for gold in flat_golden:
    for gp in gold:
        gold_clauses.append(gp['clause'])

# Quick reflection of the golden single predicates
gold_predicates = []
for gold in flat_golden:
    for gp in gold:
        gold_predicates.append(gp['predicate'])
gold_single_preds = pd.Series([pred[0]['lemma'] for pred in  filter(lambda x: len(x)==1,gold_predicates)])
gold_single_preds.value_counts()[0:10]


[pred for pred in gold_predicates if  len(pred)>2]

# Quick reflection of the parsed single predicates
parsed_predicates = []
for sent_parse in flat_parsed:
    for parse in sent_parse:
        parsed_predicates.append(parse['predicate'])
parsed_single_preds = pd.Series([pred[0]['lemma'] for pred in  filter(lambda x: len(x)==1,parsed_predicates)])
parsed_single_preds.value_counts()[0:10]

know         76
decide       34
say          32
ask          22
tell         17
think        17
wonder       17
see          15
determine    14
discuss      12
Name: count, dtype: int64

In [49]:
# Parsed predicates with more than 4 items (many more than the golden set)
list(pred for pred in parsed_predicates if len(pred)>4)

[[{'str': 'says', 'lemma': 'say', 'POS': 'VERB'},
  {'str': 'much', 'lemma': 'much', 'POS': 'ADJ'},
  {'str': 'has', 'lemma': 'have', 'POS': 'AUX'},
  {'str': 'enjoyed', 'lemma': 'enjoy', 'POS': 'VERB'},
  {'str': 'working', 'lemma': 'work', 'POS': 'VERB'},
  {'str': 'for', 'lemma': 'for', 'POS': 'ADP'}],
 [{'str': "'s", 'lemma': 'be', 'POS': 'AUX'},
  {'str': 'impossible', 'lemma': 'impossible', 'POS': 'ADJ'},
  {'str': 'find', 'lemma': 'find', 'POS': 'VERB'},
  {'str': 'out', 'lemma': 'out', 'POS': 'ADP'},
  {'str': 'half', 'lemma': 'half', 'POS': 'ADJ'},
  {'str': 'is', 'lemma': 'be', 'POS': 'AUX'}],
 [{'str': 'provided', 'lemma': 'provide', 'POS': 'VERB'},
  {'str': 'by', 'lemma': 'by', 'POS': 'ADP'},
  {'str': 'on', 'lemma': 'on', 'POS': 'ADP'},
  {'str': 'much', 'lemma': 'much', 'POS': 'ADJ'},
  {'str': 'public', 'lemma': 'public', 'POS': 'ADJ'},
  {'str': 'should', 'lemma': 'should', 'POS': 'AUX'},
  {'str': 'charge', 'lemma': 'charge', 'POS': 'VERB'}],
 [{'str': 'care', 'lemma'

In [48]:
# Parsed predicates with more then one verb per embedding predicate (more than in the golden set)
list(pred for pred in parsed_predicates if (lambda x: len([pr for pr in pred if pr['POS'] == 'VERB'])>1)(pred))


[[{'str': 'says', 'lemma': 'say', 'POS': 'VERB'},
  {'str': 'much', 'lemma': 'much', 'POS': 'ADJ'},
  {'str': 'has', 'lemma': 'have', 'POS': 'AUX'},
  {'str': 'enjoyed', 'lemma': 'enjoy', 'POS': 'VERB'},
  {'str': 'working', 'lemma': 'work', 'POS': 'VERB'},
  {'str': 'for', 'lemma': 'for', 'POS': 'ADP'}],
 [{'str': 'comment', 'lemma': 'comment', 'POS': 'VERB'},
  {'str': 'on', 'lemma': 'on', 'POS': 'ADP'},
  {'str': 'planning', 'lemma': 'plan', 'POS': 'VERB'}],
 [{'str': 'fall', 'lemma': 'fall', 'POS': 'VERB'},
  {'str': 'depending', 'lemma': 'depend', 'POS': 'VERB'},
  {'str': 'on', 'lemma': 'on', 'POS': 'ADP'}],
 [{'str': 'fall', 'lemma': 'fall', 'POS': 'VERB'},
  {'str': 'depending', 'lemma': 'depend', 'POS': 'VERB'},
  {'str': 'on', 'lemma': 'on', 'POS': 'ADP'}],
 [{'str': 'fall', 'lemma': 'fall', 'POS': 'VERB'},
  {'str': 'depending', 'lemma': 'depend', 'POS': 'VERB'},
  {'str': 'on', 'lemma': 'on', 'POS': 'ADP'}],
 [{'str': 'provided', 'lemma': 'provide', 'POS': 'VERB'},
  {'str'